In [106]:
import twint
import pandas as pd
import numpy as np
import seaborn as sns
import re
from matplotlib import pyplot as plt
from datetime import datetime
from textblob import TextBlob
from sqlalchemy import create_engine


import nest_asyncio
nest_asyncio.apply() 

In [5]:
c = twint.Config()
c.Geo = '9.0820,8.6753,923768km'
c.Limit=50000
c.Lang = 'en'
c.Since='2022-12-05'
c.Until='2022-12-07'
#c.Hide_output = True 
c.Store_csv = True
c.Output = "tweets_01.csv"
   
# searching for tweets
print('Searching tweets..')
# twint.run.Search(c)
print('Search Completed!')

Searching tweets..
Search Completed!


In [2]:
df = pd.read_csv("tweets_01.csv")
pd.set_option('display.max_columns', None) # display all columns
df.head()

id      conversation_id  \
0  1602090852742238209  1602090852742238209   
1  1602090852330897409  1602090852330897409   
2  1602090851160952832  1602000450982658049   
3  1602090850816856064  1602090850816856064   
4  1602090850808373248  1602090850808373248   

                                          created_at        date      time  \
0  2022-12-12 00:59:59 W. Central Africa Standard...  2022-12-12  00:59:59   
1  2022-12-12 00:59:59 W. Central Africa Standard...  2022-12-12  00:59:59   
2  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   
3  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   
4  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   

   timezone              user_id         username                    name  \
0       100  1536823096862248960       amoalamama                   KENNY   
1       100   751748121953202176  naijaseriousguy  OG Mikano Atikulated 💯   
2       100   976888933727522818  king_star_king_      kirito's jelly | 📦   
3       100   903924770340823040      chouechouee            👅larie lary👅   
4       100           4611969917         folymax_   Oluwatosin Folorunsho   

  place                                              tweet language  \
0   NaN  Bullshit 😤 Since you wanted more Fame to the l...       en   
1   NaN  Stephen Curry GSW  $48m p.a. Seth Curry Brookl...       en   
2   NaN  @IconzNFT_ dope thread, can't wait to see what...       en   
3   NaN                                   Ou Mariah carrey       ht   
4   NaN                                 Wisdom for life☑️💯       en   

                                            mentions urls photos  \
0  [{'screen_name': 'frenchmontana', 'name': 'fre...   []     []   
1                                                 []   []     []   
2                                                 []   []     []   
3                                                 []   []     []   
4                                                 []   []     []   

   replies_count  retweets_count  likes_count hashtags cashtags  \
0              0               0            4       []       []   
1              0               0            1       []       []   
2              0               0            0       []       []   
3              0               0            0       []       []   
4              0               0            1       []       []   

                                                link  retweet  \
0  https://twitter.com/Amoalamama/status/16020908...    False   
1  https://twitter.com/Naijaseriousguy/status/160...    False   
2  https://twitter.com/king_star_king_/status/160...    False   
3  https://twitter.com/chouechouee/status/1602090...    False   
4  https://twitter.com/folymax_/status/1602090850...    False   

                                           quote_url  video thumbnail  near  \
0  https://twitter.com/notcrypticno/status/160200...      0       NaN   NaN   
1                                                NaN      0       NaN   NaN   
2                                                NaN      0       NaN   NaN   
3  https://twitter.com/firdawslfc/status/15088141...      0       NaN   NaN   
4  https://twitter.com/pathforwisdom/status/16017...      0       NaN   NaN   

                      geo  source  user_rt_id  user_rt  retweet_id  \
0  9.0820,8.6753,923768km     NaN         NaN      NaN         NaN   
1  9.0820,8.6753,923768km     NaN         NaN      NaN         NaN   
2  9.0820,8.6753,923768km     NaN         NaN      NaN         NaN   
3  9.0820,8.6753,923768km     NaN         NaN      NaN         NaN   
4  9.0820,8.6753,923768km     NaN         NaN      NaN         NaN   

                                            reply_to  retweet_date  translate  \
0                                                 []           NaN        NaN   
1                                                 []           NaN        NaN   
2  [{'screen_name': 'IconzNFT_', 'na

In [3]:
df.shape

(93951, 36)

In [10]:
# check for missing values in all the columns
df.isnull().sum()

id                     0
conversation_id        0
created_at             0
date                   0
time                   0
timezone               0
user_id                0
username               0
name                  33
place              93863
tweet                  0
language               0
mentions               0
urls                   0
photos                 0
replies_count          0
retweets_count         0
likes_count            0
hashtags               0
cashtags               0
link                   0
retweet                0
quote_url          85215
video                  0
thumbnail          84319
near               93951
geo                    0
source             93951
user_rt_id         93951
user_rt            93951
retweet_id         93951
reply_to               0
retweet_date       93951
translate          93951
trans_src          93951
trans_dest         93951
dtype: int64

There are null values in the following columns (place, quote_url, thumbnail, near, source, user_rt_id, user_rt, retweet_id, retweet_date, translate, trans_src, and trans_dest), so I'm going to check if the missing values in them are above 70% so I can drop them. And if less than 30%, then I will fill with Not Available.

In [12]:
# checking the percentage of the missing values in all the columns
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

column_name  percent_missing
id                            id         0.000000
conversation_id  conversation_id         0.000000
created_at            created_at         0.000000
date                        date         0.000000
time                        time         0.000000
timezone                timezone         0.000000
user_id                  user_id         0.000000
username                username         0.000000
name                        name         0.035125
place                      place        99.906334
tweet                      tweet         0.000000
language                language         0.000000
mentions                mentions         0.000000
urls                        urls         0.000000
photos                    photos         0.000000
replies_count      replies_count         0.000000
retweets_count    retweets_count         0.000000
likes_count          likes_count         0.000000
hashtags                hashtags         0.000000
cashtags                cashtags         0.000000
link                        link         0.000000
retweet                  retweet         0.000000
quote_url              quote_url        90.701536
video                      video         0.000000
thumbnail              thumbnail        89.747847
near                        near       100.000000
geo                          geo         0.000000
source                    source       100.000000
user_rt_id            user_rt_id       100.000000
user_rt                  user_rt       100.000000
retweet_id            retweet_id       100.000000
reply_to                reply_to         0.000000
retweet_date        retweet_date       100.000000
translate              translate       100.000000
trans_src              trans_src       100.000000
trans_dest            trans_dest       100.000000

This shows that the following columns (place, quote_url, thumbnail, near, source, user_rt_id, user_rt, retweet_id, retweet_date, translate, trans_src, and trans_dest) has missing values above 70% and will be dropped

In [57]:
# dropping the columns
# cols = ['place', 'quote_url', 'thumbnail', 'near', 'source', 'user_rt_id', 
#         'user_rt', 'retweet_id', 'retweet_date', 'translate', 'trans_src', 'trans_dest']
# df.drop(cols, axis=1, inplace=True)


In [18]:
df.head()

id      conversation_id  \
0  1602090852742238209  1602090852742238209   
1  1602090852330897409  1602090852330897409   
2  1602090851160952832  1602000450982658049   
3  1602090850816856064  1602090850816856064   
4  1602090850808373248  1602090850808373248   

                                          created_at        date      time  \
0  2022-12-12 00:59:59 W. Central Africa Standard...  2022-12-12  00:59:59   
1  2022-12-12 00:59:59 W. Central Africa Standard...  2022-12-12  00:59:59   
2  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   
3  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   
4  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   

   timezone              user_id         username                    name  \
0       100  1536823096862248960       amoalamama                   KENNY   
1       100   751748121953202176  naijaseriousguy  OG Mikano Atikulated 💯   
2       100   976888933727522818  king_star_king_      kirito's jelly | 📦   
3       100   903924770340823040      chouechouee            👅larie lary👅   
4       100           4611969917         folymax_   Oluwatosin Folorunsho   

                                               tweet language  \
0  Bullshit 😤 Since you wanted more Fame to the l...       en   
1  Stephen Curry GSW  $48m p.a. Seth Curry Brookl...       en   
2  @IconzNFT_ dope thread, can't wait to see what...       en   
3                                   Ou Mariah carrey       ht   
4                                 Wisdom for life☑️💯       en   

                                            mentions urls photos  \
0  [{'screen_name': 'frenchmontana', 'name': 'fre...   []     []   
1                                                 []   []     []   
2                                                 []   []     []   
3                                                 []   []     []   
4                                                 []   []     []   

   replies_count  retweets_count  likes_count hashtags cashtags  \
0              0               0            4       []       []   
1              0               0            1       []       []   
2              0               0            0       []       []   
3              0               0            0       []       []   
4              0               0            1       []       []   

                                                link  retweet  video  \
0  https://twitter.com/Amoalamama/status/16020908...    False      0   
1  https://twitter.com/Naijaseriousguy/status/160...    False      0   
2  https://twitter.com/king_star_king_/status/160...    False      0   
3  https://twitter.com/chouechouee/status/1602090...    False      0   
4  https://twitter.com/folymax_/status/1602090850...    False      0   

                      geo                                           reply_to  
0  9.0820,8.6753,923768km                                                 []  
1  9.0820,8.6753,923768km                                                 []  
2  9.0820,8.6753,923768km  [{'screen_name': 'IconzNFT_', 'name': 'IconZ',...  
3  9.0820,8.6753,923768km                                                 []  
4  9.0820,8.6753,923768km                                                 []

In [47]:
print('NaN' in df['link'].values) # checking if this value is contained in the column

False


In [48]:
print('[]' in df['link'].values) # checking if this value is contained in the column

False


In [49]:
# Treating columns with [] and NaN
df.reply_to.value_counts()
# checking their percentage as well if they need to be dropped too


[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             29312
[{'screen_name': 'Naija_PR', 'name': 'Naija', 'id': '1901298962'}]                                                                                                                                                                                                                                                                                                                                                                                                                                               87

After checking, I have to
* drop url, mentions, photos, hashtags, cashtags coulmns and
* replace [] in reply_to with 'Not Available' 

In [59]:
# dropping the columns
# df.drop(['urls', 'mentions', 'photos', 'hashtags', 'cashtags'], axis = 1, inplace = True) df.shape

In [61]:
df.head()

id      conversation_id  \
0  1602090852742238209  1602090852742238209   
1  1602090852330897409  1602090852330897409   
2  1602090851160952832  1602000450982658049   
3  1602090850816856064  1602090850816856064   
4  1602090850808373248  1602090850808373248   

                                          created_at        date      time  \
0  2022-12-12 00:59:59 W. Central Africa Standard...  2022-12-12  00:59:59   
1  2022-12-12 00:59:59 W. Central Africa Standard...  2022-12-12  00:59:59   
2  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   
3  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   
4  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   

   timezone              user_id         username                    name  \
0       100  1536823096862248960       amoalamama                   KENNY   
1       100   751748121953202176  naijaseriousguy  OG Mikano Atikulated 💯   
2       100   976888933727522818  king_star_king_      kirito's jelly | 📦   
3       100   903924770340823040      chouechouee            👅larie lary👅   
4       100           4611969917         folymax_   Oluwatosin Folorunsho   

                                               tweet language  replies_count  \
0  Bullshit 😤 Since you wanted more Fame to the l...       en              0   
1  Stephen Curry GSW  $48m p.a. Seth Curry Brookl...       en              0   
2  @IconzNFT_ dope thread, can't wait to see what...       en              0   
3                                   Ou Mariah carrey       ht              0   
4                                 Wisdom for life☑️💯       en              0   

   retweets_count  likes_count  \
0               0            4   
1               0            1   
2               0            0   
3               0            0   
4               0            1   

                                                link  retweet  video  \
0  https://twitter.com/Amoalamama/status/16020908...    False      0   
1  https://twitter.com/Naijaseriousguy/status/160...    False      0   
2  https://twitter.com/king_star_king_/status/160...    False      0   
3  https://twitter.com/chouechouee/status/1602090...    False      0   
4  https://twitter.com/folymax_/status/1602090850...    False      0   

                      geo                                           reply_to  
0  9.0820,8.6753,923768km                                                 []  
1  9.0820,8.6753,923768km                                                 []  
2  9.0820,8.6753,923768km  [{'screen_name': 'IconzNFT_', 'name': 'IconZ',...  
3  9.0820,8.6753,923768km                                                 []  
4  9.0820,8.6753,923768km                                                 []

In [63]:
# replace [] in reply_to with 'Not Available'
df['reply_to'] = df['reply_to'].replace(['[]'],'Not Available')

In [65]:
df.reply_to.value_counts()

Not Available                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  29312
[{'screen_name': 'Naija_PR', 'name': 'Naija', 'id': '1901298962'}]                                                                                                                                                                                                                                                                                                                                                                                                                                               87

In [62]:
# Restricting Data to English language only
df["language"].unique() # checking if there are other languages in the dataset 

array(['en', 'ht', 'und', 'qme', 'ar', 'qam', 'art', 'pt', 'fr', 'ja',
       'qst', 'nl', 'pl', 'tr', 'es', 'zxx', 'tl', 'in', 'et', 'ca', 'da',
       'eu', 'it', 'no', 'ro', 'uk', 'de', 'qht', 'hi', 'sl', 'fi', 'sv',
       'lv', 'ko', 'hu', 'zh', 'te', 'cy', 'is', 'el', 'cs', 'lt', 'vi',
       'mr', 'ru', 'fa', 'th', 'am', 'ur', 'ne'], dtype=object)

In [66]:
# Dropping other languages that are not English
df = df[df["language"].isin(['en'])] # removing rows that are not in English

In [68]:
df.shape

(67482, 19)

In [69]:
df.head()

id      conversation_id  \
0  1602090852742238209  1602090852742238209   
1  1602090852330897409  1602090852330897409   
2  1602090851160952832  1602000450982658049   
4  1602090850808373248  1602090850808373248   
7  1602090849327874048  1602087228955824128   

                                          created_at        date      time  \
0  2022-12-12 00:59:59 W. Central Africa Standard...  2022-12-12  00:59:59   
1  2022-12-12 00:59:59 W. Central Africa Standard...  2022-12-12  00:59:59   
2  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   
4  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   
7  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   

   timezone              user_id         username                    name  \
0       100  1536823096862248960       amoalamama                   KENNY   
1       100   751748121953202176  naijaseriousguy  OG Mikano Atikulated 💯   
2       100   976888933727522818  king_star_king_      kirito's jelly | 📦   
4       100           4611969917         folymax_   Oluwatosin Folorunsho   
7       100  1270026359964946433     praiz_esther               Babygirl🌹   

                                               tweet language  replies_count  \
0  Bullshit 😤 Since you wanted more Fame to the l...       en              0   
1  Stephen Curry GSW  $48m p.a. Seth Curry Brookl...       en              0   
2  @IconzNFT_ dope thread, can't wait to see what...       en              0   
4                                 Wisdom for life☑️💯       en              0   
7           @GucciStarboi Thank God. Now I can sleep       en              0   

   retweets_count  likes_count  \
0               0            4   
1               0            1   
2               0            0   
4               0            1   
7               0            1   

                                                link  retweet  video  \
0  https://twitter.com/Amoalamama/status/16020908...    False      0   
1  https://twitter.com/Naijaseriousguy/status/160...    False      0   
2  https://twitter.com/king_star_king_/status/160...    False      0   
4  https://twitter.com/folymax_/status/1602090850...    False      0   
7  https://twitter.com/Praiz_esther/status/160209...    False      0   

                      geo                                           reply_to  
0  9.0820,8.6753,923768km                                      Not Available  
1  9.0820,8.6753,923768km                                      Not Available  
2  9.0820,8.6753,923768km  [{'screen_name': 'IconzNFT_', 'name': 'IconZ',...  
4  9.0820,8.6753,923768km                                      Not Available  
7  9.0820,8.6753,923768km  [{'screen_name': 'GucciStarboi', 'name': 'STAR...

In [71]:
# cleaning my text in the tweet column to return only text without any symbol or hashtags
def clean_text(text): 
    pat1 = r'@[^ ]+'                   #@signs
    pat2 = r'https?://[A-Za-z0-9./]+'  #links
    pat3 = r'\'s'                      #floating s's
    pat4 = r'\#\w+'                     # hashtags
    pat5 = r'&amp '
    pat6 = r'[^A-Za-z\s]'         #remove non-alphabet
    combined_pat = r'|'.join((pat1, pat2,pat3,pat4,pat5, pat6))
    text = re.sub(combined_pat,"",text).lower()
    return text.strip() 

In [81]:
df["tweet"] = df["tweet"].apply(clean_text)

C:\Users\STUTERN\AppData\Local\Temp\ipykernel_1588\965488315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tweet"] = df["tweet"].apply(clean_text)


In [82]:
df.head(3)

id      conversation_id  \
0  1602090852742238209  1602090852742238209   
1  1602090852330897409  1602090852330897409   
2  1602090851160952832  1602000450982658049   

                                          created_at        date      time  \
0  2022-12-12 00:59:59 W. Central Africa Standard...  2022-12-12  00:59:59   
1  2022-12-12 00:59:59 W. Central Africa Standard...  2022-12-12  00:59:59   
2  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   

   timezone              user_id         username                    name  \
0       100  1536823096862248960       amoalamama                   KENNY   
1       100   751748121953202176  naijaseriousguy  OG Mikano Atikulated 💯   
2       100   976888933727522818  king_star_king_      kirito's jelly | 📦   

                                               tweet language  replies_count  \
0  bullshit  since you wanted more fame to the li...       en              0   
1  stephen curry gsw  m pa seth curry brooklyn ne...       en              0   
2  dope thread cant wait to see what yall have in...       en              0   

   retweets_count  likes_count  \
0               0            4   
1               0            1   
2               0            0   

                                                link  retweet  video  \
0  https://twitter.com/Amoalamama/status/16020908...    False      0   
1  https://twitter.com/Naijaseriousguy/status/160...    False      0   
2  https://twitter.com/king_star_king_/status/160...    False      0   

                      geo                                           reply_to  
0  9.0820,8.6753,923768km                                      Not Available  
1  9.0820,8.6753,923768km                                      Not Available  
2  9.0820,8.6753,923768km  [{'screen_name': 'IconzNFT_', 'name': 'IconZ',...

In [87]:
df["tweet"][0:5]

0    bullshit  since you wanted more fame to the li...
1    stephen curry gsw  m pa seth curry brooklyn ne...
2    dope thread cant wait to see what yall have in...
4                                      wisdom for life
7                            thank god now i can sleep
Name: tweet, dtype: object

In [88]:
# Sentiment Analysis

print("Running sentiment process...")

# creating two new columns(polarity and subjectivity)
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

df["Subjectivity"] = df['tweet'].apply(getSubjectivity)
df["Polarity"] = df['tweet'].apply(getPolarity)

Running sentiment process...


C:\Users\STUTERN\AppData\Local\Temp\ipykernel_1588\1077991953.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Subjectivity"] = df['tweet'].apply(getSubjectivity)
C:\Users\STUTERN\AppData\Local\Temp\ipykernel_1588\1077991953.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Polarity"] = df['tweet'].apply(getPolarity)


In [89]:
df[['Polarity','tweet']]

Polarity                                              tweet
0      0.270833  bullshit  since you wanted more fame to the li...
1      0.000000  stephen curry gsw  m pa seth curry brooklyn ne...
2      0.000000  dope thread cant wait to see what yall have in...
4      0.000000                                    wisdom for life
7      0.000000                          thank god now i can sleep
...         ...                                                ...
93946  0.000000                    i need this as gift of the year
93947  0.000000                france vs france potentially goshhh
93948  0.000000                           sense will not kill sani
93949  0.650000  campaign after election you sure your mama dey...
93950  0.000000       im not even closei never chop this night sef

[67482 rows x 2 columns]

In [92]:
#creating a column to show if the tweet is positive, negative or neutral
def analysis(score):
    if score < 0:
        return "Negative"
    elif score == 0:
        return "Neutral"
    else:
        return "Positive"

df['Analysis'] = df['Polarity'].apply(analysis)

C:\Users\STUTERN\AppData\Local\Temp\ipykernel_1588\2513124036.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Analysis'] = df['Polarity'].apply(analysis)


In [93]:
df.head()

id      conversation_id  \
0  1602090852742238209  1602090852742238209   
1  1602090852330897409  1602090852330897409   
2  1602090851160952832  1602000450982658049   
4  1602090850808373248  1602090850808373248   
7  1602090849327874048  1602087228955824128   

                                          created_at        date      time  \
0  2022-12-12 00:59:59 W. Central Africa Standard...  2022-12-12  00:59:59   
1  2022-12-12 00:59:59 W. Central Africa Standard...  2022-12-12  00:59:59   
2  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   
4  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   
7  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   

   timezone              user_id         username                    name  \
0       100  1536823096862248960       amoalamama                   KENNY   
1       100   751748121953202176  naijaseriousguy  OG Mikano Atikulated 💯   
2       100   976888933727522818  king_star_king_      kirito's jelly | 📦   
4       100           4611969917         folymax_   Oluwatosin Folorunsho   
7       100  1270026359964946433     praiz_esther               Babygirl🌹   

                                               tweet language  replies_count  \
0  bullshit  since you wanted more fame to the li...       en              0   
1  stephen curry gsw  m pa seth curry brooklyn ne...       en              0   
2  dope thread cant wait to see what yall have in...       en              0   
4                                    wisdom for life       en              0   
7                          thank god now i can sleep       en              0   

   retweets_count  likes_count  \
0               0            4   
1               0            1   
2               0            0   
4               0            1   
7               0            1   

                                                link  retweet  video  \
0  https://twitter.com/Amoalamama/status/16020908...    False      0   
1  https://twitter.com/Naijaseriousguy/status/160...    False      0   
2  https://twitter.com/king_star_king_/status/160...    False      0   
4  https://twitter.com/folymax_/status/1602090850...    False      0   
7  https://twitter.com/Praiz_esther/status/160209...    False      0   

                      geo                                           reply_to  \
0  9.0820,8.6753,923768km                                      Not Available   
1  9.0820,8.6753,923768km                                      Not Available   
2  9.0820,8.6753,923768km  [{'screen_name': 'IconzNFT_', 'name': 'IconZ',...   
4  9.0820,8.6753,923768km                                      Not Available   
7  9.0820,8.6753,923768km  [{'screen_name': 'GucciStarboi', 'name': 'STAR...   

   Subjectivity  Polarity  Analysis  
0           0.5  0.270833  Positive  
1           0.0  0.000000   Neutral  
2           0.0  0.000000   Neutral  
4           0.0  0.000000   Neutral  
7           0.0  0.000000   Neutral

In [96]:
df[['Polarity','tweet','Analysis']]

Polarity                                              tweet  Analysis
0      0.270833  bullshit  since you wanted more fame to the li...  Positive
1      0.000000  stephen curry gsw  m pa seth curry brooklyn ne...   Neutral
2      0.000000  dope thread cant wait to see what yall have in...   Neutral
4      0.000000                                    wisdom for life   Neutral
7      0.000000                          thank god now i can sleep   Neutral
...         ...                                                ...       ...
93946  0.000000                    i need this as gift of the year   Neutral
93947  0.000000                france vs france potentially goshhh   Neutral
93948  0.000000                           sense will not kill sani   Neutral
93949  0.650000  campaign after election you sure your mama dey...  Positive
93950  0.000000       im not even closei never chop this night sef   Neutral

[67482 rows x 3 columns]

In [103]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\STUTERN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [104]:
# # removing stopwords from the tweets

stop = stopwords.words('english')
df['tweet_without_stopwords'] = df['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df.head()

id      conversation_id  \
0  1602090852742238209  1602090852742238209   
1  1602090852330897409  1602090852330897409   
2  1602090851160952832  1602000450982658049   
4  1602090850808373248  1602090850808373248   
7  1602090849327874048  1602087228955824128   

                                          created_at        date      time  \
0  2022-12-12 00:59:59 W. Central Africa Standard...  2022-12-12  00:59:59   
1  2022-12-12 00:59:59 W. Central Africa Standard...  2022-12-12  00:59:59   
2  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   
4  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   
7  2022-12-12 00:59:58 W. Central Africa Standard...  2022-12-12  00:59:58   

   timezone              user_id         username                    name  \
0       100  1536823096862248960       amoalamama                   KENNY   
1       100   751748121953202176  naijaseriousguy  OG Mikano Atikulated 💯   
2       100   976888933727522818  king_star_king_      kirito's jelly | 📦   
4       100           4611969917         folymax_   Oluwatosin Folorunsho   
7       100  1270026359964946433     praiz_esther               Babygirl🌹   

                                               tweet language  replies_count  \
0  bullshit  since you wanted more fame to the li...       en              0   
1  stephen curry gsw  m pa seth curry brooklyn ne...       en              0   
2  dope thread cant wait to see what yall have in...       en              0   
4                                    wisdom for life       en              0   
7                          thank god now i can sleep       en              0   

   retweets_count  likes_count  \
0               0            4   
1               0            1   
2               0            0   
4               0            1   
7               0            1   

                                                link  retweet  video  \
0  https://twitter.com/Amoalamama/status/16020908...    False      0   
1  https://twitter.com/Naijaseriousguy/status/160...    False      0   
2  https://twitter.com/king_star_king_/status/160...    False      0   
4  https://twitter.com/folymax_/status/1602090850...    False      0   
7  https://twitter.com/Praiz_esther/status/160209...    False      0   

                      geo                                           reply_to  \
0  9.0820,8.6753,923768km                                      Not Available   
1  9.0820,8.6753,923768km                                      Not Available   
2  9.0820,8.6753,923768km  [{'screen_name': 'IconzNFT_', 'name': 'IconZ',...   
4  9.0820,8.6753,923768km                                      Not Available   
7  9.0820,8.6753,923768km  [{'screen_name': 'GucciStarboi', 'name': 'STAR...   

   Subjectivity  Polarity  Analysis  \
0           0.5  0.270833  Positive   
1           0.0  0.000000   Neutral   
2           0.0  0.000000   Neutral   
4           0.0  0.000000   Neutral   
7           0.0  0.000000   Neutral   

                             tweet_without_stopwords  
0  bullshit since wanted fame little got could do...  
1   stephen curry gsw pa seth curry brooklyn nets pa  
2               dope thread cant wait see yall store  
4                                        wisdom life  
7                                    thank god sleep

In [107]:
df[['Polarity','tweet','Analysis', 'tweet_without_stopwords']]

Polarity                                              tweet  Analysis  \
0      0.270833  bullshit  since you wanted more fame to the li...  Positive   
1      0.000000  stephen curry gsw  m pa seth curry brooklyn ne...   Neutral   
2      0.000000  dope thread cant wait to see what yall have in...   Neutral   
4      0.000000                                    wisdom for life   Neutral   
7      0.000000                          thank god now i can sleep   Neutral   
...         ...                                                ...       ...   
93946  0.000000                    i need this as gift of the year   Neutral   
93947  0.000000                france vs france potentially goshhh   Neutral   
93948  0.000000                           sense will not kill sani   Neutral   
93949  0.650000  campaign after election you sure your mama dey...  Positive   
93950  0.000000       im not even closei never chop this night sef   Neutral   

                                 tweet_without_stopwords  
0      bullshit since wanted fame little got could do...  
1       stephen curry gsw pa seth curry brooklyn nets pa  
2                   dope thread cant wait see yall store  
4                                            wisdom life  
7                                        thank god sleep  
...                                                  ...  
93946                                     need gift year  
93947                france vs france potentially goshhh  
93948                                    sense kill sani  
93949         campaign election sure mama dey proud bayi  
93950                im even closei never chop night sef  

[67482 rows x 4 columns]

In [108]:
# saving my dataframe
df.to_csv("C:/Users/STUTERN/Desktop/Intermediate_project/week_11_project_dataframe.csv")

The dataframe will be imported into Power Bi for Visualization

In [109]:
df.shape

(67482, 23)